### RAG TEST

In [18]:
query_list = [
    """{
    "purpose": "침실에 안정감을 주는 식물",
    "preferred_style": "내추럴",
    "preferred_color": "#6B8E23",
    "plant_type": "관엽식물",
    "season": "가을",
    "humidity": "높음",
    "has_dog": true,
    "has_cat": false,
    "isAirCond": false,
    "watering_frequency": "주 2회",
    "user_experience": "초보자",
    "emotion": "편안하고 따뜻한 분위기"
  }""",
  """
    {
    "purpose": "거실 공간을 화사하게 만들기",
    "preferred_style": "북유럽",
    "preferred_color": "#F2E9D8",
    "plant_type": "관엽식물",
    "season": "봄",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": true,
    "isAirCond": true,
    "watering_frequency": "주 1회",
    "user_experience": "초보자",
    "emotion": "밝고 포근한 분위기"
  }
""",
"""
{
    "purpose": "주방에 잘 어울리는 식물",
    "preferred_style": "빈티지",
    "preferred_color": "#C97D63",
    "plant_type": "허브",
    "season": "여름",
    "humidity": "높음",
    "has_dog": false,
    "has_cat": false,
    "isAirCond": false,
    "watering_frequency": "주 3회",
    "user_experience": "중급자",
    "emotion": "따뜻하고 향긋한 느낌"
  }
""",
"""
{
    "purpose": "카페 한켠 분위기 연출",
    "preferred_style": "내추럴",
    "preferred_color": "#D6C7A1",
    "plant_type": "관엽식물",
    "season": "봄",
    "humidity": "높음",
    "has_dog": false,
    "has_cat": false,
    "isAirCond": false,
    "watering_frequency": "주 2회",
    "user_experience": "전문가",
    "emotion": "자연스럽고 싱그러운 분위기"
  }
""",
"""
{
    "purpose": "아이 방 공기 정화용",
    "preferred_style": "심플",
    "preferred_color": "#E1F5C4",
    "plant_type": "공기정화식물",
    "season": "겨울",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": true,
    "isAirCond": true,
    "watering_frequency": "주 1회",
    "user_experience": "초보자",
    "emotion": "맑고 깨끗한 분위기"
  }
""",
"""
{
    "purpose": "집들이 선물용 식물",
    "preferred_style": "심플",
    "preferred_color": "#E4E8D2",
    "plant_type": "관엽식물",
    "season": "봄",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": true,
    "isAirCond": true,
    "watering_frequency": "주 1회",
    "user_experience": "초보자",
    "emotion": "부담 없고 산뜻한 느낌"
  }
""",
"""
{
    "purpose": "부모님께 드릴 건강한 느낌의 선물 식물",
    "preferred_style": "내추럴",
    "preferred_color": "#C2D7B5",
    "plant_type": "공기정화식물",
    "season": "사계절",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": false,
    "isAirCond": false,
    "watering_frequency": "주 1회",
    "user_experience": "초보자",
    "emotion": "깨끗하고 편안한 분위기"
  }
""",
"""
{
    "purpose": "친구 생일 선물용 꽃 식물",
    "preferred_style": "포인트",
    "preferred_color": "#F7D7D0",
    "plant_type": "꽃식물",
    "season": "봄",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": false,
    "isAirCond": true,
    "watering_frequency": "주 2회",
    "user_experience": "중급자",
    "emotion": "화사하고 기분 좋은 느낌"
  }
""",
"""
{
    "purpose": "회사 동료에게 줄 관리 쉬운 선물용 식물",
    "preferred_style": "모던",
    "preferred_color": "#B4C6A6",
    "plant_type": "다육식물",
    "season": "사계절",
    "humidity": "낮음",
    "has_dog": false,
    "has_cat": false,
    "isAirCond": true,
    "watering_frequency": "2주 1회",
    "user_experience": "초보자",
    "emotion": "담백하고 세련된 느낌"
  }
""",
"""
{
    "purpose": "연인에게 특별한 의미를 담아 주는 식물",
    "preferred_style": "클래식",
    "preferred_color": "#DEA6A0",
    "plant_type": "꽃식물",
    "season": "겨울",
    "humidity": "보통",
    "has_dog": false,
    "has_cat": true,
    "isAirCond": false,
    "watering_frequency": "주 1회",
    "user_experience": "중급자",
    "emotion": "따뜻하고 로맨틱한 분위기"
  }
"""
]

In [19]:
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from dotenv import load_dotenv 
import os 

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "plant-recommend" 
index = pc.Index(index_name) 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

response_list = []

for query in query_list:
    query_embeddings = embeddings.embed_query(query)        
    query_response = \
    response = index.query(
        top_k = 3,                 # 결과 반환 갯수
        include_values=False,      # 벡터 임베딩 반환 여부
        include_metadata=True,     # 메타 데이터 반환 여부
        vector=query_embeddings    # 검색할 벡터 임베딩
    )
    response_list.append(response)

In [20]:
# LLM 평가 함수
from langchain_openai import ChatOpenAI

def test_llm(query, response):
    prompt = f"""
    너는 RAG 시스템의 응답 품질을 평가하는 전문 LLM 평가자(Evaluator)다.
    주어진 query와 RAG를 통해 생성된 response를 비교하여, 응답의 정확성, 관련성, 완전성을 기반으로 점수를 매겨라.

    ### 평가 기준 ###
    1. 정확성 Accuracy (0~1)
        - response가 query의 의도와 의미를 정확히 반영하는 정도
    2. 관련성 Relevance (0~1)
        - response 내용이 query와 얼마나 밀접하게 연결되는지
    3. 완전성 Completeness (0~1)
        - query에 필요한 정보가 누락되지 않고 충분히 포함되었는지
    4. 최종 평가 (0~1)
        - 1, 2, 3 기준을 종합해 내린 평가

    ### 평가 대상 ###
    query = {query}
    response = {response}
"""
        
    model = ChatOpenAI(
        model='gpt-4o',
        temperature=1
    )
    result = model.invoke([prompt])
    
    return result

In [21]:
for response in response_list:
    print(response['matches'][0].metadata)
    print(response['matches'][0].score)

    print(response['matches'][1].metadata)
    print(response['matches'][1].score)

    print(response['matches'][2].metadata)
    print(response['matches'][2].score)

{'dataNo': '343', 'difficulty': '쉬움', 'fContent': '꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다.', 'fDay': '8', 'fGrow': '5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다.', 'fMonth': '12', 'fType': "돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.", 'fUse': '다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다.', 'flowLang': '인기, 설레임', 'flowNm': '칼랑코에', 'imgUrl1': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-1.jpg', 'imgUrl2': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-2.jpg', 'imgUrl3': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-3.jpg', 'interior': '칼랑코에는 밝은 조명이 있는 창가에 두어 공간에 생기를 불어넣을 수 있으며, 다양한 색상의 꽃이 피어나면서 시각적인 포인트를 제공합니다. 화분은 모던한 스타일의 화분을 선택해 조화를 이루도록 합니다.', 'isAirCond': True, 'isToxicToCat': False, 'isToxicToDog': False, 'style': '모던, 미니멀리즘, 보헤미안', 'text': "인기, 설레임, 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은

In [28]:
res_list = []

for query, response in zip(query_list, response_list):
    res = test_llm(query, response['matches'][0].metadata)
    res_list.append(res)

    res = test_llm(query, response['matches'][1].metadata)
    res_list.append(res)
    
    res = test_llm(query, response['matches'][2].metadata)
    res_list.append(res)

In [29]:
for i, res in enumerate(res_list):
    print(i+1)
    print(res.content)
    print()
    print()

1
### 평가

1. **정확성 (Accuracy): 0.4**
   - Query에서는 침실에 안정감을 주는 식물을 찾고 있으며, 초보자에게 적합하고, 물주기는 주 2회 정도를 원하는 반면, Response에서는 '칼랑코에'라는 식물을 추천하고 있습니다. 하지만 이 식물은 주 1회 물주기를 권장하며, 스타일 또한 모던, 미니멀리즘, 보헤미안을 고수하여 사용자 요구사항의 '내추럴 스타일'과 일치하지 않습니다. 전체적인 조건 중 일부는 맞지만, 정확한 요구사항 반영이 부족합니다.

2. **관련성 (Relevance): 0.7**
   - '칼랑코에'가 관엽식물이며, 강아지에게 안전하다는 조건을 만족시키기는 하지만, 침실에 필요한 '편안하고 따뜻한 분위기'와는 거리가 있습니다. 또한 스타일이 '모던, 미니멀리즘, 보헤미안'에 더 가까워 관련성이 다소 낮습니다. 사용자의 조건 대부분을 맞추지 못하고 있습니다.

3. **완전성 (Completeness): 0.5**
   - 제공된 정보에는 식물의 생김새와 성장 조건 등에 관한 정보가 많지만, 사용자 요구의 각 요소를 완전히 충족시키지는 못하고 있습니다. 특히, 물주기 빈도와 관련된 정보가 사용자의 요구와 상충됩니다. 이러한 불일치는 완전성을 떨어뜨립니다.

4. **최종 평가 (Overall): 0.5**
   - 전반적으로, 사용자의 요청에 완전히 부합하지 못하며, 제공된 정보가 부분적으로만 관련성이 있는 것으로 보입니다. 여러 조건 중 일부는 만족하지만, 완전히 필요를 충족시키지 못하고, 특히 스타일과 물주기 빈도가 맞지 않는 점이 주목됩니다. 이상의 평가를 종합해 보았을 때 중간 수준의 점수를 부여하게 됩니다.


2
### 평가 ###

1. 정확성 Accuracy (0.3/1)
   - Query에서 사용자가 원하는 식물은 "침실에 안정감을 주는 식물"이며, 이는 "편안하고 따뜻한 분위기"와 내추럴한 스타일을 선호합니다. 그러나 응답에서는 식물 '칼라'가 "환희, 열정"이라는 꽃말을 갖고 있으며, 이는